# Pycle demo 0: a minimal working example example
This notebook provides a quick of Compressive Learning with the `pycle` toolbox. It demonstrates the typical workflow with `pycle` and core features. In particular, we will:
0. Generate a synthetic dataset
1. Sketch it, by first generating our sketch operator
2. Use this sketch to estimate the centroids associated with the dataset

We assume that you have installed `pycle` with `pip` or that you have the `pycle` folder in the same directory as this notebook.

In [ ]:
# General imports
import numpy as np
import matplotlib.pyplot as plt

# Fix the random seed for reproducibility
np.random.seed(42)

## PART 0: generate a dataset with utils

To demonstrate the toolbox, we generate a simple synthetic dataset, from a Gaussian mixture model in two dimensions.

In [ ]:
from pycle.utils.datasets import generatedataset_GMM

d = 2     # Dimension
K = 4     # Number of Gaussians
n = 20000 # Number of samples we want to generate
# We use the generatedataset_GMM method from pycle (we ask that the entries are <= 1, and imbalanced clusters)
X = generatedataset_GMM(d,K,n,normalize='l_inf-unit-ball',balanced=False) 

# Bounds on the dataset, necessary for compressive k-means
bounds = np.array([-np.ones(d),np.ones(d)]) # We assumed the data is normalized between -1 and 1

# Visualize the dataset
plt.figure(figsize=(5,5))
plt.title("Full dataset")
plt.scatter(X[:,0],X[:,1],s=1, alpha=0.15)
plt.show()

## PART 1: sketching the dataset
We first compress the dataset as a sketch. All the tools required for this step are located in the `pycle.sketching` submodule (which we import under the handier alias `sk`). 

In [ ]:
from pycle.sketching.frequency_sampling import drawFrequencies
from pycle.sketching.feature_maps.MatrixFeatureMap import MatrixFeatureMap
from pycle.sketching import computeSketch

Recall that the sketch of a dataset $X$ is given by

$$ \boldsymbol z = \sum_{\boldsymbol x_i \in X} \Phi(\boldsymbol x_i) $$

We first generate the feature map $\Phi$. In this simple example, we chose here random Fourier features $\Phi(\boldsymbol x) = \exp(\mathrm{i}\Omega^T\boldsymbol x)$, which is the usual choice for k-means and GMM fitting. In this expression, $\Omega$ is a matrix of $m$ "frequency" vectors, generated from $\boldsymbol \omega_j \sim \Lambda$ (without entering into the details, in this case we pick a Folded Gaussian distribution parametrized by $\Sigma$, which should roughly represent the covariance of the desired clusters). The sketch size $m$ should here be of the order of the number of parameters to estimate, here the number of centroid coordinates, $Kd$.

In [ ]:
# Pick the dimension m: 5*K*d is usually (just) enough in clustering (here m = 50)
m = 10*K*d 

# For this simple example, assume we have a priori a rough idea of the size of the clusters
Sigma = 0.1*np.eye(d)

# According to the Folded Gaussian rule, we want m frequencies in dimension d, parametrized by Sigma
Omega = drawFrequencies("FoldedGaussian",d,m,Sigma)

# The feature map is a standard one, the complex exponential of projections on Omega^T
Phi = MatrixFeatureMap("ComplexExponential",Omega)

# And sketch X with Phi: we map a 20000x2 dataset -> a 50-dimensional complex vector
z = computeSketch(X,Phi)

## PART 2: compressive learning from the sketch
Now that we have computed `z`, we want to learn from this sketch alone. Algorithms that learn from the sketch are  available in the `pycle.compressive_learning` submodule (which we simply refer to as `cl`). 

In [ ]:
from pycle.compressive_learning.CLOMP_CKM import CLOMP_CKM

To solve compressive k-means (CKM) with the CLOMP-R algorithm, we create an instance of `cl.CLOMP_CKM`, to which we specify the feature map, the number of centroids we want, the data bounds, and finally the dataset sketch `z`. We then tell the solver to search for the centroids by calling `fit_once()`, and access the found solution as `current_sol` (which returns a tuple with the relative weigths of the clusters, and then the centroids themselves).

In [ ]:
# Initialize the solver object
ckm_solver = CLOMP_CKM(Phi,K=K,bounds=bounds,sketch=z)

# Launch the CLOMP optimization procedure
ckm_solver.fit_once()

# Get the solution
(weights,centroids) = ckm_solver.current_sol

In [ ]:
# Visualize the centroids (we re-use the dataset for visual comparison)
plt.figure(figsize=(5,5))
plt.title("Compressively learned centroids")
plt.scatter(X[:,0],X[:,1],s=1, alpha=0.15)
plt.scatter(centroids[:,0],centroids[:,1],s=1000*weights)
plt.legend(["Data","Centroids"])
plt.show()

from pycle.utils.metrics import SSE
print("SSE: {}".format(SSE(X,centroids)))

In the figure above, the size of the orange centroid markers is proportional to the weights of the centroids.
As you can see, thanks to `pycle` only a few lines of code suffice to start doing compressive learning. In the next demos, we show how to use some other features of the toolbox, such as automatically setting $\Sigma$, solving other tasks (GMM), and a privacy-preservation layer on top of the sketch, and using quantized sketch contributions.